In [41]:
import torch
import torch.nn as nn
import pandas as pd
import random

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3060'

In [42]:
# Hyperparameters
block_size = 256
batch_size = 64
max_iters  = 5000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embed = 384
n_head = 6
n_layers = 6
dropout = 0.2

In [43]:
rdata = pd.read_csv('animes.csv')
rdata = list(rdata['synopsis'])
rdata = [str(i).replace('\r', '').replace('\t', '').split('\n')[0] for i in rdata]
rdata = [f" {i}" for i in rdata]
rdata = [ bytes(i, 'utf-8').decode('utf-8', 'ignore') for i in rdata]


vocab = sorted(list(set(''.join(rdata).replace('.', ''))))
stoi = {s:i+1 for i, s in enumerate(vocab)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

#data = torch.tensor([encode(i) for i in rdata], dtype=torch.long)
edata = encode(''.join(rdata))
data = torch.tensor(edata, dtype=torch.long)
train_data = data[:int(0.9*len(edata))]
val_data   = data[int(0.9*len(edata)):]

In [45]:
def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [46]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class Head(nn.Module):
    